In [7]:
#Скрипт для проверки работы виртуального окружения
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Основная страница Лента.ру
BASE_URL = 'https://lenta.ru'

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def simple_parser():

    #Составила словарь категорий для обращения
    categories = {
        'economics': 'https://lenta.ru/rubrics/economics/',
        'science': 'https://lenta.ru/rubrics/science/',
        'culture': 'https://lenta.ru/rubrics/culture/',
        'sport': 'https://lenta.ru/rubrics/sport/'
    }

    all_news = []

    for category, url in categories.items():
        print(f"\nОбрабатываю категорию: {category}")

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        links = []
        for a in soup.find_all('a', href=True):
            href = a['href']
            if '/news/' in href or '/articles/' in href:
                if href.startswith('/'):
                    full_url = 'https://lenta.ru' + href
                else:
                    full_url = href

                if full_url not in links:
                    links.append(full_url)

        links = links[:25]
        print(f"Найдено {len(links)} ссылок")

        for i, link in enumerate(links):
            print(f"  {i+1}/{len(links)}: {link[:60]}...")

            try:
                news_response = requests.get(link, timeout=10)
                news_soup = BeautifulSoup(news_response.text, 'html.parser')

                title = news_soup.find('h1')
                if title:
                    title_text = title.text.strip()
                else:
                    continue

                content = news_soup.find('div', class_='topic-body__content')
                if content:
                    text = content.get_text(strip=True)
                else:
                    continue

                all_news.append({
                    'title': title_text,
                    'text': text,
                    'category': category,
                    'url': link
                })

                time.sleep(0.5)

            except Exception as e:
                print(f"    Ошибка: {e}")
                continue

    if all_news:
        df = pd.DataFrame(all_news)
        df.to_csv('simple_lenta_news.csv', index=False)
        print(f"\nСохранено {len(df)} новостей")
        return df
    else:
        print("\nНе удалось собрать новости")
        return None

simple_parser()


Обрабатываю категорию: economics
Найдено 25 ссылок
  1/25: https://lenta.ru/articles/2026/01/17/china/...
  2/25: https://lenta.ru/news/2026/01/22/postsovetskaya-strana-stolk...
  3/25: https://lenta.ru/news/2026/01/22/moskvichey-predupredili-o-s...
  4/25: https://lenta.ru/news/2026/01/22/sber-predstavil-pervogo-v-r...
  5/25: https://lenta.ru/news/2026/01/22/oboydutsya/...
  6/25: https://lenta.ru/news/2026/01/22/v-rossii-rezko-vyros-ob-em-...
  7/25: https://lenta.ru/news/2026/01/22/lukashenko-prizval-delat-vs...
  8/25: https://lenta.ru/news/2026/01/22/buryatiya-temp/...
  9/25: https://lenta.ru/news/2026/01/22/v-rossiyskom-gorode-pod-ezd...
  10/25: https://lenta.ru/news/2026/01/22/rossiyane-rasskazali-ob-isp...
  11/25: https://lenta.ru/news/2026/01/22/rynkam-smartfonov-i-kompyut...
  12/25: https://lenta.ru/news/2026/01/22/raznitsu-temperatur-v-tsent...
  13/25: https://lenta.ru/news/2026/01/22/rukhnul/...
  14/25: https://lenta.ru/articles/2026/01/13/rzd/...
  15/25: https://l

,title,text,category,url
0,«Для них это проект бессмертия» Миллиардеры из...,Китайские миллиардеры стали стремиться к созда...,economics,https://lenta.ru/articles/2026/01/17/china/
1,Постсоветская страна столкнулась с обрушением ...,В 2025 годуКиргизиястолкнулась с падением эксп...,economics,https://lenta.ru/news/2026/01/22/postsovetskay...
2,Москвичей предупредили о сильных снегопадах,В конце января вМосквеожидаются сильные снегоп...,economics,https://lenta.ru/news/2026/01/22/moskvichey-pr...
3,Сбер представил первого в России ИИ-агента для...,В 2026 годуСбервыведет на рынок уникального ИИ...,economics,https://lenta.ru/news/2026/01/22/sber-predstav...
4,Рост стоимости подарков для учителей признали ...,Правительство не поддержало законопроект депут...,economics,https://lenta.ru/news/2026/01/22/oboydutsya/
...,...,...,...,...
84,Футболиста ПСЖ обвинили в торговле людьми,Защитника ПСЖ и сборнойФранцииЛукаса Эрнандеса...,sport,https://lenta.ru/news/2026/01/21/futbolista-ps...
85,Россиянин стал миллионером благодаря голу на 9...,Клиент букмекерской конторы выиграл миллион ру...,sport,https://lenta.ru/news/2026/01/21/rossiyanin-st...
86,В Латвии вместо выступления россиян на Олимпиа...,Латвийские телеканалы планируют транслировать ...,sport,https://lenta.ru/news/2026/01/21/v-latvii-vmes...
87,Олимпийский чемпион раскритиковал отсутствие т...,Четырехкратный олимпийский чемпион по биатлону...,sport,https://lenta.ru/news/2026/01/21/olimpiyskiy-c...
